<a href="https://colab.research.google.com/github/ttogle918/AI_practice/blob/main/%EC%8A%B9%ED%8C%A8%EC%98%88%EC%B8%A1/01_%ED%96%89%EB%8F%99%EB%8D%B0%EC%9D%B4%ED%84%B0_event%ED%8A%B9%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[데이터](https://dacon.io/competitions/official/235583/overview/description) : 스타크래프트 데이터

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
from tqdm import tqdm                       # 진행바
import warnings
warnings.filterwarnings("ignore")           # 경고 문구 미표시

# 파일 load

In [3]:
file_path = '/content/drive/MyDrive/Colab Notebooks/dataset/dacon/행동 데이터/235583_행동 데이터 분석 인공지능 AI 경진대회_data.zip'
dir_path = '/content/competition_data'

In [4]:
import zipfile
from zipfile import ZipFile

with zipfile.ZipFile(f'{file_path}', 'r') as obj:
  namelist = obj.namelist()
  obj.extractall(dir_path)
namelist

['sample_submission.csv', 'test.csv', 'train.csv']

# dataframe 재구조화 및 전체 데이터 불러오기

p_key : (game_id, winner)
독립 변수 : 나머지 ( list 형태 )

| game_id | winner | time	| player | species | event |	event_contents |


### 전체 데이터 불러오기

In [5]:
# arg2 == None 일 때 arg1은 game_id값, arg2 == 0 or 1일 때 arg1은 dataframe 형태
def get_dict_data(arg1, arg2=None) :
  global train_data
#  if arg2 == None :
  return train_data[(train_data['game_id'] == arg1)].drop(columns=['game_id', 'winner', 'species']).to_dict('list')

#  return train_data[(train_data['game_id'] == arg1) & (train_data['player'] == arg2)].drop(columns=['game_id', 'winner', 'player', 'species']).to_dict('list')

In [6]:
df = pd.DataFrame(columns=['game_id', 'winner', 'data', 'p0_data', 'p1_data'])

def preprocessing(data) :
  global df, cnt

  df_temp = data[['game_id', 'winner', 'species', 'player']].drop_duplicates()
  df_temp = df_temp.pivot(index=['game_id', 'winner'], columns='player', values='species').reset_index().set_axis(['game_id', 'winner', 'p0_species', 'p1_species'], axis=1)
  df_temp['data'] = df_temp['game_id'].apply(get_dict_data)

  # 이전 행과 같은 경기인지 비교
  if len(df) == 0 or df['game_id'].iloc[-1] != df_temp['game_id'].iloc[0] :
    # 다르다면 => df concat
    df = pd.concat([df, df_temp])
  else :
    # 같다면 => 마지막 행의 dict와 concat 후 데이터프레임 concat
    for k in df_temp['data'].iloc[-1].keys() :
      df['data'].iloc[-1][k].extend(df_temp['data'].iloc[0][k])

    df = pd.concat([df, df_temp.iloc[1:]])

  print(f'--{cnt}번쨰-- 합친 df 길이 : {len(df)}   새로 포함된 df 길이 : {len(df_temp)}')

  del df_temp

In [7]:
for cnt, train_data in enumerate(pd.read_csv(dir_path + '/train.csv', chunksize=1024*1024)) :
  preprocessing(train_data)
  if cnt == 40 :
    break


--0번쨰-- 합친 df 길이 : 605   새로 포함된 df 길이 : 605
--1번쨰-- 합친 df 길이 : 1226   새로 포함된 df 길이 : 622
--2번쨰-- 합친 df 길이 : 1830   새로 포함된 df 길이 : 605
--3번쨰-- 합친 df 길이 : 2425   새로 포함된 df 길이 : 596
--4번쨰-- 합친 df 길이 : 3032   새로 포함된 df 길이 : 608
--5번쨰-- 합친 df 길이 : 3647   새로 포함된 df 길이 : 616
--6번쨰-- 합친 df 길이 : 4265   새로 포함된 df 길이 : 619
--7번쨰-- 합친 df 길이 : 4876   새로 포함된 df 길이 : 612
--8번쨰-- 합친 df 길이 : 5503   새로 포함된 df 길이 : 628
--9번쨰-- 합친 df 길이 : 6107   새로 포함된 df 길이 : 605
--10번쨰-- 합친 df 길이 : 6711   새로 포함된 df 길이 : 605
--11번쨰-- 합친 df 길이 : 7305   새로 포함된 df 길이 : 595
--12번쨰-- 합친 df 길이 : 7893   새로 포함된 df 길이 : 589
--13번쨰-- 합친 df 길이 : 8502   새로 포함된 df 길이 : 610
--14번쨰-- 합친 df 길이 : 9108   새로 포함된 df 길이 : 607
--15번쨰-- 합친 df 길이 : 9723   새로 포함된 df 길이 : 616
--16번쨰-- 합친 df 길이 : 10324   새로 포함된 df 길이 : 602
--17번쨰-- 합친 df 길이 : 10921   새로 포함된 df 길이 : 598
--18번쨰-- 합친 df 길이 : 11519   새로 포함된 df 길이 : 599
--19번쨰-- 합친 df 길이 : 12135   새로 포함된 df 길이 : 617
--20번쨰-- 합친 df 길이 : 12730   새로 포함된 df 길이 : 596
--21번쨰-- 합친 df 길이 : 13337   새로 포함된 df 길이

In [8]:
# 마지막행은 불완전함. => 제거
df = df.iloc[:-1]

In [9]:
import gc
gc.collect()

0

In [10]:
df.head()

,game_id,winner,data,p0_data,p1_data,p0_species,p1_species
0,0,1,"{'time': [0.0, 0.0, 0.02, 0.02, 0.14, 0.14, 0....",NaN,NaN,T,T
1,1,1,"{'time': [0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0...",NaN,NaN,P,T
2,2,0,"{'time': [0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0....",NaN,NaN,P,Z
3,3,0,"{'time': [0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0....",NaN,NaN,T,P
4,4,0,"{'time': [0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0...",NaN,NaN,T,Z


## 전체 데이터로 추출할 특징

전체 데이터로 추출할 특징

- 종족
- 클릭 시간 평균
- 총 플레이 시간(마지막 데이터 시간)


In [11]:
df['play_time'] = df['data'].apply(lambda x : x['time'][-1])

In [12]:
df['time_avg'] = df['data'].apply(lambda x : (sum(x['time'][1:]) - sum(x['time'][:-1])) / (len(x['time'])-1) )
df.head()

,game_id,winner,data,p0_data,p1_data,p0_species,p1_species,play_time,time_avg
0,0,1,"{'time': [0.0, 0.0, 0.02, 0.02, 0.14, 0.14, 0....",NaN,NaN,T,T,7.24,0.006356
1,1,1,"{'time': [0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0...",NaN,NaN,P,T,9.59,0.003942
2,2,0,"{'time': [0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0....",NaN,NaN,P,Z,9.59,0.004318
3,3,0,"{'time': [0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0....",NaN,NaN,T,P,9.59,0.003186
4,4,0,"{'time': [0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0...",NaN,NaN,T,Z,5.43,0.003360


## player1과 player0 데이터 각각 추출

각 플레이어의 특징
- 카메라 조작까지의 시간
- 클릭 시간 평균
- 소환한 시간(전체, 각각)
- 위치 거리(첫번쨰 지점에서부터 거리의 평균)


In [13]:
df['p0_data'] = df['data'].apply(lambda x : pd.DataFrame(x)[pd.DataFrame(x)['player'] == 0].to_dict('list'))
df['p1_data'] = df['data'].apply(lambda x : pd.DataFrame(x)[pd.DataFrame(x)['player'] == 1].to_dict('list'))

In [22]:
# count
df['time_cnt'] = df['data'].apply(lambda x : pd.DataFrame(x)['time'].count() )
df['p0_time_cnt'] = df['p0_data'].apply(lambda x : pd.DataFrame(x)['time'].count() )
df['p1_time_cnt'] = df['p1_data'].apply(lambda x : pd.DataFrame(x)['time'].count() )

df['p0_time_cnt_div_all_cnt'] = df['p0_time_cnt'] / df['time_cnt']
df['p1_time_cnt_div_all_cnt'] = df['p1_time_cnt'] / df['time_cnt']
df['p1_time_cnt_div_all_cnt_diff'] = df['p0_time_cnt_div_all_cnt'] - df['p1_time_cnt_div_all_cnt']

# std
df['p0_time_std'] = df['p0_data'].apply(lambda x : pd.DataFrame(x)['time'].std() )
df['p1_time_std'] = df['p1_data'].apply(lambda x : pd.DataFrame(x)['time'].std() )
df['time_std_diff'] = df['p0_time_std'] - df['p1_time_std']

In [23]:
df.head(1)

,game_id,winner,data,p0_data,p1_data,p0_species,p1_species,play_time,time_avg,p0dic_event,...,p1SetControlGroup,p0_time_cnt,time_cnt,p1_time_cnt,p0_time_cnt_div_all_cnt,p1_time_cnt_div_all_cnt,p1_time_cnt_div_all_cnt_diff,p0_time_std,p1_time_std,time_std_diff
0,0,1,"{'time': [0.0, 0.0, 0.02, 0.02, 0.14, 0.14, 0....","{'time': [0.0, 0.02, 0.02, 0.14, 0.14, 0.14, 0...","{'time': [0.0, 0.33, 0.34, 0.34, 0.34, 0.35, 0...",T,T,7.24,0.006356,"{'Camera': 444, 'Selection': 50, 'Right Click'...",...,1,592,1140,548,0.519298,0.480702,0.038596,2.052092,1.696707,0.355385


In [76]:
import re
# 1. 알파벳, 띄어쓰기만 남기기
# 2. 한글자, 대문자로만 이루어진 단어 삭제 ==> 16진수 표현식일 가능성
def get_words_in_event_contents(x) :
  x = re.sub(r"[^a-z^A-Z^ ]", "", str(x))
  x = x.split(' ')
  i = 0
  while i < len(x) :
    if x[i].isupper() or len(x[i]) == 0 :   # 대문자이거나 값이 없을 때 삭제
      del x[i]
    else :
      i += 1
  return x

def get_dic_in_event_contents(x) :   # 수정 필요
  temp = pd.DataFrame(x)
  temp['info'] = temp['event_contents'].apply(get_words_in_event_contents)
  dic_ = dict()
  for s in sum(temp['info'].to_list(), []) :
    dic_[s] = dic_[s]+1 if s in dic_ else 1
  return dic_

def get_dic_in_event(x) :
  temp = pd.DataFrame(x)
  return temp['event'].value_counts().to_dict()

event : 행동 종류

event_contents : 행동 상세

1) Ability : 생산, 공격 등 선수의 주요 행동

2) AddToControlGroup : 부대에 추가

3) Camera : 시점 선택

4) ControlGroup : 부대 행동

5) GetControlGroup : 부대 불러오기

6) Right Click : 마우스 우클릭

7) Selection : 객체 선택

8) SetControlGroup : 부대 지정

In [25]:
df['p0dic_event'] = df['p0_data'].apply(get_dic_in_event)
df['p1dic_event'] = df['p1_data'].apply(get_dic_in_event)
df.head()

,game_id,winner,data,p0_data,p1_data,p0_species,p1_species,play_time,time_avg,p0dic_event,...,p1SetControlGroup,p0_time_cnt,time_cnt,p1_time_cnt,p0_time_cnt_div_all_cnt,p1_time_cnt_div_all_cnt,p1_time_cnt_div_all_cnt_diff,p0_time_std,p1_time_std,time_std_diff
0,0,1,"{'time': [0.0, 0.0, 0.02, 0.02, 0.14, 0.14, 0....","{'time': [0.0, 0.02, 0.02, 0.14, 0.14, 0.14, 0...","{'time': [0.0, 0.33, 0.34, 0.34, 0.34, 0.35, 0...",T,T,7.24,0.006356,"{'Camera': 444, 'Selection': 50, 'Right Click'...",...,1,592,1140,548,0.519298,0.480702,0.038596,2.052092,1.696707,0.355385
1,1,1,"{'time': [0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0...","{'time': [0.0, 0.01, 0.01, 0.02, 0.02, 0.02, 0...","{'time': [0.0, 0.0, 0.01, 0.01, 0.02, 0.02, 0....",P,T,9.59,0.003942,"{'Camera': 627, 'Selection': 186, 'GetControlG...",...,8,1223,2434,1211,0.502465,0.497535,0.004930,2.912545,2.816285,0.096260
2,2,0,"{'time': [0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0....","{'time': [0.0, 0.01, 0.01, 0.02, 0.02, 0.02, 0...","{'time': [0.0, 0.01, 0.01, 0.02, 0.02, 0.02, 0...",P,Z,9.59,0.004318,"{'Camera': 413, 'Right Click': 160, 'GetContro...",...,9,851,2222,1371,0.382988,0.617012,-0.234023,3.043779,2.797152,0.246628
3,3,0,"{'time': [0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0....","{'time': [0.0, 0.01, 0.01, 0.01, 0.01, 0.02, 0...","{'time': [0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0...",T,P,9.59,0.003186,"{'Camera': 713, 'Right Click': 276, 'Selection...",...,19,1389,3011,1622,0.461309,0.538691,-0.077383,2.852661,2.846988,0.005673
4,4,0,"{'time': [0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0...","{'time': [0.0, 0.01, 0.01, 0.01, 0.02, 0.02, 0...","{'time': [0.0, 0.0, 0.01, 0.01, 0.02, 0.02, 0....",T,Z,5.43,0.003360,"{'Camera': 430, 'GetControlGroup': 224, 'Right...",...,8,966,1617,651,0.597403,0.402597,0.194805,1.490782,1.651671,-0.160889


In [26]:
for s in ['Ability', 'AddToControlGroup', 'Camera', 'ControlGroup', 'GetControlGroup', 'Right Click', 'Selection', 'SetControlGroup'] :
  df['p0'+s] = df['p0dic_event'].apply(lambda x : x[s] if s in x else 0)
  df['p1'+s] = df['p1dic_event'].apply(lambda x : x[s] if s in x else 0)
df.head()

,game_id,winner,data,p0_data,p1_data,p0_species,p1_species,play_time,time_avg,p0dic_event,...,p1SetControlGroup,p0_time_cnt,time_cnt,p1_time_cnt,p0_time_cnt_div_all_cnt,p1_time_cnt_div_all_cnt,p1_time_cnt_div_all_cnt_diff,p0_time_std,p1_time_std,time_std_diff
0,0,1,"{'time': [0.0, 0.0, 0.02, 0.02, 0.14, 0.14, 0....","{'time': [0.0, 0.02, 0.02, 0.14, 0.14, 0.14, 0...","{'time': [0.0, 0.33, 0.34, 0.34, 0.34, 0.35, 0...",T,T,7.24,0.006356,"{'Camera': 444, 'Selection': 50, 'Right Click'...",...,1,592,1140,548,0.519298,0.480702,0.038596,2.052092,1.696707,0.355385
1,1,1,"{'time': [0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0...","{'time': [0.0, 0.01, 0.01, 0.02, 0.02, 0.02, 0...","{'time': [0.0, 0.0, 0.01, 0.01, 0.02, 0.02, 0....",P,T,9.59,0.003942,"{'Camera': 627, 'Selection': 186, 'GetControlG...",...,8,1223,2434,1211,0.502465,0.497535,0.004930,2.912545,2.816285,0.096260
2,2,0,"{'time': [0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0....","{'time': [0.0, 0.01, 0.01, 0.02, 0.02, 0.02, 0...","{'time': [0.0, 0.01, 0.01, 0.02, 0.02, 0.02, 0...",P,Z,9.59,0.004318,"{'Camera': 413, 'Right Click': 160, 'GetContro...",...,9,851,2222,1371,0.382988,0.617012,-0.234023,3.043779,2.797152,0.246628
3,3,0,"{'time': [0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0....","{'time': [0.0, 0.01, 0.01, 0.01, 0.01, 0.02, 0...","{'time': [0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0...",T,P,9.59,0.003186,"{'Camera': 713, 'Right Click': 276, 'Selection...",...,19,1389,3011,1622,0.461309,0.538691,-0.077383,2.852661,2.846988,0.005673
4,4,0,"{'time': [0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.0...","{'time': [0.0, 0.01, 0.01, 0.01, 0.02, 0.02, 0...","{'time': [0.0, 0.0, 0.01, 0.01, 0.02, 0.02, 0....",T,Z,5.43,0.003360,"{'Camera': 430, 'GetControlGroup': 224, 'Right...",...,8,966,1617,651,0.597403,0.402597,0.194805,1.490782,1.651671,-0.160889


### model 사용 ==> baseline RF 56%

In [28]:
df['p0_species'].value_counts()

T    9659
Z    8280
P    6963
Name: p0_species, dtype: int64

In [29]:
# T는 Z P 둘 다 0일 때 ==> 더미변수
df['p0_species_Z'] = -1
df['p0_species_P'] = -1

df['p0_species_Z'].loc[df['p0_species'] == "Z"] = 1
df['p0_species_P'].loc[df['p0_species'] == "Z"] = 0
df['p0_species_P'].loc[df['p0_species'] == "P"] = 1
df['p0_species_Z'].loc[df['p0_species'] == "P"] = 0

df['p0_species_P'].loc[df['p0_species'] == "T"] = 0
df['p0_species_Z'].loc[df['p0_species'] == "T"] = 0


df['p1_species_Z'] = -1
df['p1_species_P'] = -1

df['p1_species_Z'].loc[df['p1_species'] == "Z"] = 1
df['p1_species_P'].loc[df['p1_species'] == "Z"] = 0
df['p1_species_P'].loc[df['p1_species'] == "P"] = 1
df['p1_species_Z'].loc[df['p1_species'] == "P"] = 0

df['p1_species_P'].loc[df['p1_species'] == "T"] = 0
df['p1_species_Z'].loc[df['p1_species'] == "T"] = 0

print(df['p0_species_P'].value_counts())
print(df['p0_species_Z'].value_counts(), '\n')
print(df['p1_species_P'].value_counts())
print(df['p1_species_Z'].value_counts())

0    17939
1     6963
Name: p0_species_P, dtype: int64
0    16622
1     8280
Name: p0_species_Z, dtype: int64 

0    17882
1     7020
Name: p1_species_P, dtype: int64
0    16761
1     8141
Name: p1_species_Z, dtype: int64


In [31]:
print(df.winner.value_counts())
df['winner'] = df['winner'].astype(int)
df.winner.dtypes

1    12518
0    12384
Name: winner, dtype: int64


dtype('int64')

In [30]:
df.dtypes

game_id                          object
winner                           object
data                             object
p0_data                          object
p1_data                          object
p0_species                       object
p1_species                       object
play_time                       float64
time_avg                        float64
p0dic_event                      object
p1dic_event                      object
p0Ability                         int64
p1Ability                         int64
p0AddToControlGroup               int64
p1AddToControlGroup               int64
p0Camera                          int64
p1Camera                          int64
p0ControlGroup                    int64
p1ControlGroup                    int64
p0GetControlGroup                 int64
p1GetControlGroup                 int64
p0Right Click                     int64
p1Right Click                     int64
p0Selection                       int64
p1Selection                       int64


In [32]:
df.columns

Index(['game_id', 'winner', 'data', 'p0_data', 'p1_data', 'p0_species',
       'p1_species', 'play_time', 'time_avg', 'p0dic_event', 'p1dic_event',
       'p0Ability', 'p1Ability', 'p0AddToControlGroup', 'p1AddToControlGroup',
       'p0Camera', 'p1Camera', 'p0ControlGroup', 'p1ControlGroup',
       'p0GetControlGroup', 'p1GetControlGroup', 'p0Right Click',
       'p1Right Click', 'p0Selection', 'p1Selection', 'p0SetControlGroup',
       'p1SetControlGroup', 'p0_time_cnt', 'time_cnt', 'p1_time_cnt',
       'p0_time_cnt_div_all_cnt', 'p1_time_cnt_div_all_cnt',
       'p1_time_cnt_div_all_cnt_diff', 'p0_time_std', 'p1_time_std',
       'time_std_diff', 'p0_species_Z', 'p0_species_P', 'p1_species_Z',
       'p1_species_P'],
      dtype='object')

In [37]:
features = ['play_time', 'time_avg',
       'p0Ability', 'p1Ability', 'p0AddToControlGroup', 'p1AddToControlGroup',
       'p0Camera', 'p1Camera', 'p0ControlGroup', 'p1ControlGroup',
       'p0GetControlGroup', 'p1GetControlGroup', 'p0Right Click',
       'p1Right Click', 'p0Selection', 'p1Selection', 'p0SetControlGroup',
       'p1SetControlGroup', 'p0_time_cnt', 'time_cnt', 'p1_time_cnt',
       'p0_time_cnt_div_all_cnt', 'p1_time_cnt_div_all_cnt',
       'p1_time_cnt_div_all_cnt_diff', 'p0_time_std', 'p1_time_std',
       'time_std_diff', 'p0_species_Z', 'p0_species_P', 'p1_species_Z',
       'p1_species_P']

label = 'winner'

In [40]:
df[features].isna().sum()

play_time                        0
time_avg                         0
p0Ability                        0
p1Ability                        0
p0AddToControlGroup              0
p1AddToControlGroup              0
p0Camera                         0
p1Camera                         0
p0ControlGroup                   0
p1ControlGroup                   0
p0GetControlGroup                0
p1GetControlGroup                0
p0Right Click                    0
p1Right Click                    0
p0Selection                      0
p1Selection                      0
p0SetControlGroup                0
p1SetControlGroup                0
p0_time_cnt                      0
time_cnt                         0
p1_time_cnt                      0
p0_time_cnt_div_all_cnt          0
p1_time_cnt_div_all_cnt          0
p1_time_cnt_div_all_cnt_diff     0
p0_time_std                     25
p1_time_std                     36
time_std_diff                   61
p0_species_Z                     0
p0_species_P        

In [51]:
pd.DataFrame(df[df['p0_time_std'].isna()]['p0_data'].iloc[0])

,time,player,event,event_contents
0,0.0,0,Camera,"at (28.0, 20.25)"


In [53]:
df = df.dropna()
df[features].isna().sum()

play_time                       0
time_avg                        0
p0Ability                       0
p1Ability                       0
p0AddToControlGroup             0
p1AddToControlGroup             0
p0Camera                        0
p1Camera                        0
p0ControlGroup                  0
p1ControlGroup                  0
p0GetControlGroup               0
p1GetControlGroup               0
p0Right Click                   0
p1Right Click                   0
p0Selection                     0
p1Selection                     0
p0SetControlGroup               0
p1SetControlGroup               0
p0_time_cnt                     0
time_cnt                        0
p1_time_cnt                     0
p0_time_cnt_div_all_cnt         0
p1_time_cnt_div_all_cnt         0
p1_time_cnt_div_all_cnt_diff    0
p0_time_std                     0
p1_time_std                     0
time_std_diff                   0
p0_species_Z                    0
p0_species_P                    0
p1_species_Z  

In [54]:
from sklearn.model_selection import train_test_split
train, val, ytrain, yval = train_test_split(df[features], df[label], train_size=0.80, test_size=0.20, random_state=2)
len(train), len(ytrain)

(19872, 19872)

In [55]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0)
clf.fit(train, ytrain)
#clf.predict(val)


RandomForestClassifier(random_state=0)

In [56]:
clf.score(val, yval)

0.5900583618434293

### event_contents에서 추출

대표적인 것 몇가지 카운트

#### Ability 중에서 추출

In [92]:
def get_dic_in_event_contents_filter(x, sss) :   # 특정 event 내에서 만들 때, get_dic_in_event_contents 수정해서 같이 사용할 수 있게 수정해야함!
  temp__ = pd.DataFrame(x)

  dic_ = dict()
  for s in sum(temp__[temp__['event']==sss]['event_contents'].apply(get_words_in_event_contents).to_list(), []) :
    dic_[s] = dic_[s]+1 if s in dic_ else 1
  return dic_

In [93]:
df['p0dic_event_contents'] = df['p0_data'].apply(get_dic_in_event_contents_filter, args=('Ability',))
df['p1dic_event_contents'] = df['p1_data'].apply(get_dic_in_event_contents_filter, args=('Ability',))

In [94]:
cnt_event_contents_dic = {} # 전체 cnt x, 존재하는지만 확인 후 1 더하기

for dic_ in df['p0dic_event_contents'].to_list() :
  for k in dic_.keys() :
    if k in cnt_event_contents_dic :
      cnt_event_contents_dic[k] = cnt_event_contents_dic[k]+1
    else :
      cnt_event_contents_dic[k] = 1

cnt_event_contents_dic.keys()

dict_keys(['TrainSCV', 'BuildSupplyDepot', 'Location', 'Stop', 'BuildBarracks', 'BuildRefinery', 'Target', 'CreepOnlyBlockerx', 'BuildCommandCenter', 'UpgradeToOrbitalCommand', 'TrainMarine', 'BuildFactory', 'BuildBarracksReactor', 'KDCharge', 'BuildStarport', 'BuildEngineeringBay', 'BuildWidowMine', 'TrainMedivac', 'BuildFactoryTechLab', 'UpgradeTerranInfantryArmor', 'CancelSlot', 'CalldownMULE', 'None', 'TrainProbe', 'BuildPylon', 'BuildGateway', 'BuildAssimilator', 'DestructibleSignsFunny', 'CancelLast', 'BuildCyberneticsCore', 'TrainAdept', 'TrainMothershipCore', 'ResearchWarpGate', 'BuildRoboticsFacility', 'Attack', 'Patrol', 'TrainWarpPrism', 'BuildRoboticsBay', 'AdeptPhaseShift', 'ChronoBoost', 'CyberneticsCore', 'ResearchGraviticDrive', 'TransformToWarpGate', 'BuildForge', 'MothershipCorePurifyNexus', 'Pylon', 'TrainDisruptor', 'BuildNexus', 'BuildTwilightCouncil', 'UpgradeGroundWeapons', 'UnloadUnitWarpPrism', 'PurificationNovaTargeted', 'AutoTurret', 'WarpInStalker', 'UnloadT

In [95]:
temp_df = pd.DataFrame({'name':cnt_event_contents_dic.keys(), 'cnt':cnt_event_contents_dic.values()})
temp_df.describe()

,cnt
count,470.000000
mean,1174.034043
std,2618.342941
min,1.000000
25%,18.000000
50%,135.000000
75%,1086.000000
max,24637.000000


In [100]:
for s in ['Location', 'Stop', 'Target', 'None'] :
  print(temp_df[temp_df['name']==s].cnt)

2    24637
Name: cnt, dtype: int64
3    3768
Name: cnt, dtype: int64
6    23766
Name: cnt, dtype: int64
22    5428
Name: cnt, dtype: int64


In [138]:
temp_s = ''.join(temp_df['name'].to_list())
for sss in range(65, 91) : # A to Z
  temp_s = temp_s.replace(chr(sss), ' '+chr(sss))
temp_s = temp_s.split(' ')
set_temp_s = set(temp_s)
dic_temp_s = {}
for sss in set_temp_s :
  if len(sss) > 1 :
    dic_temp_s[sss] = temp_s.count(sss)
pd.DataFrame({'name':dic_temp_s.keys(), 'cnt':dic_temp_s.values()}).sort_values('cnt', ascending=False).head(20)

,name,cnt
0,Build,56
255,Train,34
181,Research,31
217,Upgrade,22
127,To,19
54,Morph,17
275,Cancel,16
253,Warp,15
349,Command,14
47,Unload,12


In [117]:
temp_df['Train'] = temp_df['name'].apply(lambda x : 'Train' in x)
temp_df['Build'] = temp_df['name'].apply(lambda x : 'Train' in x)
temp_df['Train'].value_counts()

False    436
True      34
Name: Train, dtype: int64

In [104]:
temp_df.sort_values(by='cnt', ascending=False).head(20)

,name,cnt
2,Location,24637
6,Target,23766
34,Attack,19035
27,DestructibleSignsFunny,16851
0,TrainSCV,9603
1,BuildSupplyDepot,9240
5,BuildRefinery,8599
106,MorphDrone,8206
4,BuildBarracks,8198
110,MorphOverlord,7941


In [96]:
temp_df[temp_df['cnt'] >= temp_df['cnt'].median()]

,name,cnt
0,TrainSCV,9603
1,BuildSupplyDepot,9240
2,Location,24637
3,Stop,3768
4,BuildBarracks,8198
...,...,...
302,ResearchInfernalPreIgniter,138
305,ForceField,157
313,Refinery,163
317,DestructibleIcexVertical,175


In [ ]:
def get_first_word(s)  :
  i = 1
  while i < len(s) and s[i].islower() :
    i += 1
  return s[:i]

temp_df['name_'] = temp_df['name'].apply(get_first_word)

In [20]:
df['p0_data'] = df['game_id'].apply(get_dict_data, args=(0,))

# 특징 뽑기 방법

1) Feature로 각 플레이어의 종족, 게임 내 이벤트 발생 숫자, 플레이어간 이벤트 발생 숫자의 차이를 사용합니다.  
1-1) 종족을 숫자로 변환 (T:0, P:1, Z:2)  
1-2) 이벤트 발생 숫자는 8개 이벤트에 대해 각각 추출  
1-3) 플레이어간 이벤트 발생 숫자를 Feature로 추가  
1-4) 플레이어간 이벤트 발생 숫자 차이를 Feature로 추가

In [ ]:
train_data.describe()

,game_id,winner,time,player
count,1000.0,1000.0,1000.000000,1000.000000
mean,0.0,1.0,3.489740,0.494000
std,0.0,0.0,1.654625,0.500214
min,0.0,1.0,0.000000,0.000000
25%,0.0,1.0,2.400000,0.000000
50%,0.0,1.0,3.490000,0.000000
75%,0.0,1.0,4.562500,1.000000
max,0.0,1.0,6.240000,1.000000


# column 확인